## Hoja de Trabajo 2
### Christopher Sandoval 13660

(Para probar el funcionamiento del código de una forma más sencilla se pueden correr los archivos train.py y test.py)

Importamos las librerías necesarias.

In [22]:
import numpy as np
from scipy.optimize import minimize
import time
import pickle
import math
from functools import reduce
import scipy

### FUNCIONES

Se define la función que nos ayudará a leer los datos de las imagenes. Esta función es parte del repositorio de fashion-mnist de donde se obtuvieron los datos de prueba https://github.com/zalandoresearch/fashion-mnist 

In [23]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

Se definen las funciones necesarias para realizar el entrenamiento y pruebas del modelo. Basado en el código visto en clase.

In [24]:
sigmoid = lambda x: 1.0 / (1.0 + np.exp(-x))

flatten_list_of_arrays = lambda list_of_arrays: reduce(
    lambda acc, v: np.array([*acc.flatten(), *v.flatten()]),
    list_of_arrays
)

def inflate_matrixes(flat_thetas, shapes):
    layers = len(shapes) + 1
    sizes = [shape[0] * shape[1] for shape in shapes]
    steps = np.zeros(layers, dtype=int)

    for i in range(layers - 1):
        steps[i + 1] = steps[i] + sizes[i]

    return [
        flat_thetas[steps[i]: steps[i + 1]].reshape(*shapes[i])
        for i in range(layers - 1)
    ]

def feed_forward(thetas, X):
    a = [X]
    for i in range(len(thetas)):
        a.append(
            sigmoid(
                np.matmul(
                    np.hstack((np.ones(len(X)).reshape(len(X), 1), a[i])),
                    thetas[i].T
                )
            )
        )
    return a

def back_propagation(flat_thetas, shapes, X, Y):
    m, layers = len(X), len(shapes) + 1
    thetas = inflate_matrixes(flat_thetas, shapes)
    a = feed_forward(thetas, X)
    deltas = [*range(layers - 1), a[-1] - Y]

    for i in range(layers - 2, 0, -1):
        deltas[i] = (deltas[i + 1] @ np.delete(thetas[i], 0, 1)) * (a[i] * (1 - a[i]))
        
    new_deltas = []
    for n in range(layers - 1):
        new_deltas.append(
            (deltas[n + 1].T @ np.hstack((
                np.ones(len(a[n])).reshape(len(a[n]), 1),
                a[n]
            ))) / m
        )

    new_deltas = np.asarray(new_deltas)

    return flatten_list_of_arrays(
        new_deltas
    )

def cost_function(flat_thetas, shapes, X, Y):
    a = feed_forward(
        inflate_matrixes(flat_thetas, shapes),
        X
    )
    return -(Y * np.log(a[-1]) + (1 - Y) * np.log(1 - a[-1])).sum() / len(X)

def predict(flat_thetas, shapes, X):
    thetas = inflate_matrixes(flat_thetas, shapes)
    return feed_forward(thetas, X)[-1]

### MODELO

Establecemos el factor de normalización de los datos y el modelo de nuestra red neuronal ya que deben permanecer consistentes entre el entrenamiento y las pruebas. Nuestro modelo tendra 3 capas, la capa de entrada consistirá de 784 neuronas (cada una correspondiente a un pixel de las imagenes), la capa oculta que constará de 125 neuronas, y por último la capa de salida que constará de 10 neuronas cada una correspondiente a los diferentes tipos de ropa que estamos intentando identificar.

In [25]:
NORMALIZATION_FACTOR = 1000.0

INPUT_NEURONS = 784
HIDDEN_NEURONS = 125
OUTPUT_NEURONS = 10

layers = np.array([
    INPUT_NEURONS,
    HIDDEN_NEURONS,
    EXIT_NEURONS
])
theta_shapes = np.hstack((
    layers[1:].reshape(len(layers) - 1, 1),
    (layers[:-1] + 1).reshape(len(layers) - 1, 1)
))

### ENTRENAMIENTO

Primero se establecen las iteraciones máximas permitidas en el entrenamiento.

In [32]:
MAX_ITERATIONS = 3333

Se importan los datos con la funcion mnist_reader y se normalizan los valores de X correspondientes a los pixeles de las imágenes.

In [38]:
X_train, y_train = load_mnist('data/fashion', kind='train')

X_train = X_train / NORMALIZATION_FACTOR
m, n = X_train.shape
y_train = y_train.reshape(m, 1)
y_train = (y_train == np.array(range(10))).astype(int)

flat_thetas = flatten_list_of_arrays([
    np.random.rand(*theta_shape)
    for theta_shape in theta_shapes
])

Se inicia el entrenamiento con un máximo de 3333 iteraciones.

In [ ]:
start = time.time()

print("Iniciando entrenamiento...")

result = minimize(
    fun = cost_function,
    x0 = flat_thetas,
    args = (theta_shapes, X_train, y_train),
    method = 'L-BFGS-B',
    jac = back_propagation,
    options = {'disp': False, 'maxiter': MAX_ITERATIONS}
)

end = time.time()

print("Entrenamiento completado en {}s".format(end - start))

outfile = open('trained_model', 'wb')
pickle.dump(result.x, outfile)
outfile.close()

print("Archivo del modelo creado!")

### TESTING

Para probar el modelo se utilizara el set de datos t10k. Primero importamos los datos y los normalizamos.

In [26]:
X_test, y_test = load_mnist('data/fashion', kind='t10k')
X = X_test / NORMALIZATION_FACTOR

Luego abriremos el modelo que fue guardado en el entrenamiento y lo enviamos a nuestra funcion de prediccion.

In [27]:
openfile = open("trained_model", "rb")
thetas_trained = pickle.load(openfile)

predictions = predict(thetas_trained, theta_shapes, X)

Por último convertiremos las predicciones al mismo formato de los datos de prueba y comparamos los resultados para conocer el porcentaje de precisión de nuestro modelo

In [31]:
predictions_cleaned = np.array([])

for prediction in predictions:
    condition = (prediction == np.amax(prediction))
    index = np.where(condition)[0]
    predictions_cleaned = np.concatenate((predictions_cleaned,index))

matches = np.sum(predictions_cleaned == y_test)
precision = matches/len(predictions)


print('-----------RESULTADOS TESTING-----------')
print('Predicciones acertadas: {:d}/{:d}'.format(matches, len(predictions)))
print('Porcentaje de precision: {:.2f}%'.format(precision*100))

-----------RESULTADOS TESTING-----------
Predicciones acertadas: 8458/10000
Porcentaje de precision: 84.58%


Como podemos observar se obtuvo una precisión del 84.58% prediciendo los datos del set de testing. Este dataset contiene 10000 imagenes diferentes a las que se usaron para el entrenamiento. El rendimiento del modelo es positivo y no se notó ningún incremento sustancial al probar con más iteraciones en el entrenamiento.